In [0]:
from spellchecker import SpellChecker

def checker (sent):
  updated=[]
  spell=SpellChecker('en')
  for word in sent.split():
    updated.append(spell.correction(word))
  return ' '.join(updated)
checked_sent=checker("this is jsut graet!")
print(checked_sent)

this is just great

##### Edit operation on typos (jsut & graet)
- j**s**ut first delete s => it becomes jut then insert the s =>ju**s**t
- gr**a**et first delete a=> it becomes grat then insert a=> gre**a**t

In [0]:
#importing neede libraries
from bs4 import BeautifulSoup
import re
from pyspark.sql.functions import desc
from pyspark.ml.feature import Word2Vec
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
import math
import numpy as np


In [0]:
#pre-processing function
token = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("english")
stop_words = sc.textFile("/FileStore/tables/Assignment_1/stopwords.txt").collect()

def sent_tokenizer(sent):
  '''
  PROMISES: removes numbers, tokenizes the string, remove stop words, and finds the stem of the word
  REQUIRES: sent is a string
  return: an array of processed sentence
  '''
  sent = re.sub(r'\d+', '',sent)
  
  temp=token.tokenize(sent)
  result=[]
  for word in temp:
    if word  not in stop_words:
      result.append(stemmer.stem(word))
  return result


In [0]:
#cosine similarity fucntion
def angle(v1, v2):
  '''
  PROMISES: calculates the angle between two vectors
  REQUIRES: v1 & v2 are vectors of same size
  return the angle or cosine similarity number
  '''
  return float(np.dot(v1, v2) / (np.sqrt(np.dot(v1, v1)) *np.sqrt(np.dot(v2, v2))) )


In [0]:
##Loading java.csv
java_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Java.csv",inferSchema=True,header=True,multiLine=True,escape='"')
java_RDD=java_df.select('Body','AcceptedAnswerId','Title','Score').rdd
java_RDD.map(lambda x: (x[0],x[1],x[2]))
java_RDD_filtered=java_RDD.map(lambda x:(str(BeautifulSoup(x[0]).find_all('p')),x[1],x[2],x[3]))
java_RDD_filtered=java_RDD_filtered.map(lambda x: (re.sub('<p>','',x[0]),x[1],x[2],x[3]))
java_RDD_filtered=java_RDD_filtered.map(lambda x: (re.sub('</p>','',x[0]),x[1],x[2],x[3]))
java_tokenize=java_RDD_filtered.map(lambda x: (sent_tokenizer(x[0]),x[1],sent_tokenizer(x[2]),x[3],x[0],x[2]))
java_tokenize=java_tokenize.map(lambda x: (x[0]+x[2],x[1],x[3],x[4],x[5]))
java_df_processed=spark.createDataFrame(java_tokenize).toDF('Body_Title','AcceptedAnswerId','Score','Body','Title')

#transforming Body_Title col into DTM
java_word2Vec = Word2Vec( vectorSize=50, minCount=3,inputCol="Body_Title", outputCol="Body_Title_result")
java_model = java_word2Vec.fit(java_df_processed)
java_df_DTM=java_model.transform(java_df_processed)

#filtering for un answered question with the highest score 
java_df_not_answered=java_df_DTM.filter('AcceptedAnswerId is null')
java_df_not_answered=java_df_not_answered.orderBy(desc('Score')).take(1)
java_similar_question=java_df_not_answered[0][5]

#calculating similarity of body and title based on answered question
java_df_answered=java_df_DTM.filter('AcceptedAnswerId is not null')
java_rdd_similarity=java_df_answered.rdd.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5]))
java_rdd_similarity=java_rdd_similarity.map(lambda x:(x[1],x[2],x[3],x[4],angle(x[5],java_similar_question)))

#sorting by similarity
java_df_similarity=spark.createDataFrame(java_rdd_similarity).toDF('AcceptedAnswerId','Score','Body','Title','Similarity')
java_df_similarity=java_df_similarity.orderBy(desc('Similarity')).collect()


In [0]:
##Loading py.csv
py_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Python.csv",inferSchema=True,header=True,multiLine=True,escape='"')
py_RDD=py_df.select('Body','AcceptedAnswerId','Title','Score').rdd
py_RDD.map(lambda x: (x[0],x[1],x[2]))
py_RDD_filtered=py_RDD.map(lambda x:(str(BeautifulSoup(x[0]).find_all('p')),x[1],x[2],x[3]))
py_RDD_filtered=py_RDD_filtered.map(lambda x: (re.sub('<p>','',x[0]),x[1],x[2],x[3]))
py_RDD_filtered=py_RDD_filtered.map(lambda x: (re.sub('</p>','',x[0]),x[1],x[2],x[3]))
py_tokenize=py_RDD_filtered.map(lambda x: (sent_tokenizer(x[0]),x[1],sent_tokenizer(x[2]),x[3],x[0],x[2]))
py_tokenize=py_tokenize.map(lambda x: (x[0]+x[2],x[1],x[3],x[4],x[5]))
py_df_processed=spark.createDataFrame(py_tokenize).toDF('Body_Title','AcceptedAnswerId','Score','Body','Title')

#transforming Body_Title col into DTM
py_word2Vec = Word2Vec( vectorSize=50, minCount=3,inputCol="Body_Title", outputCol="Body_Title_result")
py_model = py_word2Vec.fit(py_df_processed)
py_df_DTM=py_model.transform(py_df_processed)

#filtering for un answered question with the highest score 
py_df_not_answered=py_df_DTM.filter('AcceptedAnswerId is null')
py_df_not_answered=py_df_not_answered.orderBy(desc('Score')).take(1)
py_similar_question=py_df_not_answered[0][5]

#calculating similarity of body and title based on answered question
py_df_answered=py_df_DTM.filter('AcceptedAnswerId is not null')
py_rdd_similarity=py_df_answered.rdd.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5]))
py_rdd_similarity=py_rdd_similarity.map(lambda x:(x[1],x[2],x[3],x[4],angle(x[5],py_similar_question)))

#sorting by similarity
py_df_similarity=spark.createDataFrame(py_rdd_similarity).toDF('AcceptedAnswerId','Score','Body','Title','Similarity')
py_df_similarity=py_df_similarity.orderBy(desc('Similarity')).collect()


In [0]:
##Loading js.csv
js_df=spark.read.csv("/FileStore/tables/Assignment_1/SO_Javascript.csv",inferSchema=True,header=True,multiLine=True,escape='"')
js_RDD=js_df.select('Body','AcceptedAnswerId','Title','Score').rdd
js_RDD.map(lambda x: (x[0],x[1],x[2]))
js_RDD_filtered=js_RDD.map(lambda x:(str(BeautifulSoup(x[0]).find_all('p')),x[1],x[2],x[3]))
js_RDD_filtered=js_RDD_filtered.map(lambda x: (re.sub('<p>','',x[0]),x[1],x[2],x[3]))
js_RDD_filtered=js_RDD_filtered.map(lambda x: (re.sub('</p>','',x[0]),x[1],x[2],x[3]))
js_tokenize=js_RDD_filtered.map(lambda x: (sent_tokenizer(x[0]),x[1],sent_tokenizer(x[2]),x[3],x[0],x[2]))
js_tokenize=js_tokenize.map(lambda x: (x[0]+x[2],x[1],x[3],x[4],x[5]))
js_df_processed=spark.createDataFrame(js_tokenize).toDF('Body_Title','AcceptedAnswerId','Score','Body','Title')

#transforming Body_Title col into DTM
js_word2Vec = Word2Vec( vectorSize=50, minCount=3,inputCol="Body_Title", outputCol="Body_Title_result")
js_model = js_word2Vec.fit(js_df_processed)
js_df_DTM=js_model.transform(js_df_processed)

#filtering for un answered question with the highest score 
js_df_not_answered=js_df_DTM.filter('AcceptedAnswerId is null')
js_df_not_answered=js_df_not_answered.orderBy(desc('Score')).take(1)
js_similar_question=js_df_not_answered[0][5]

#calculating similarity of body and title based on answered question
js_df_answered=js_df_DTM.filter('AcceptedAnswerId is not null')
js_rdd_similarity=js_df_answered.rdd.map(lambda x: (x[0],x[1],x[2],x[3],x[4],x[5]))
js_rdd_similarity=js_rdd_similarity.map(lambda x:(x[1],x[2],x[3],x[4],angle(x[5],js_similar_question)))

#sorting by similarity
js_df_similarity=spark.createDataFrame(js_rdd_similarity).toDF('AcceptedAnswerId','Score','Body','Title','Similarity')
js_df_similarity=js_df_similarity.orderBy(desc('Similarity')).collect()


In [0]:
java_df_similarity=spark.createDataFrame(java_df_similarity)
py_df_similarity=spark.createDataFrame(py_df_similarity)
js_df_similarity=spark.createDataFrame(js_df_similarity)

In [0]:
print('---------Java------------')
print(" The highest scored unanswered question is: \n Score= {}\n Body= {}\nTitle= {}".format(java_df_not_answered[0][2],java_df_not_answered[0][3],java_df_not_answered[0][4]))
print("\nTop 3 similar questions with accepted answers are:\n")
java_df_similarity.show(3,truncate=True)
print('\n---------Pyhton------------')
print(" The highest scored unanswered question is: \n Score= {}\n Body= {}\nTitle= {}".format(py_df_not_answered[0][2],py_df_not_answered[0][3],py_df_not_answered[0][4]))
print("\nTop 3 similar questions with accepted answers are:\n")
py_df_similarity.show(3,truncate=True)
print('\n---------JavaScript------------')
print(" The highest scored unanswered question is: \n Score= {}\n Body= {}\nTitle= {}".format(js_df_not_answered[0][2],js_df_not_answered[0][3],js_df_not_answered[0][4]))
print("\nTop 3 similar questions with accepted answers are:\n")
js_df_similarity.show(3,truncate=True)

---------Java------------
 The highest scored unanswered question is: 
 Score= 1668
 Body= [I am relatively new to Java, and often find that I need to sort a <code>Map&lt;Key, Value&gt;</code> on the values., Since the values are not unique, I find myself converting the <code>keySet</code> into an <code>array</code>, and sorting that array through <strong>array sort</strong> with a <strong>custom comparator</strong> that sorts on the value associated with the key., Is there an easier way?]
Title= Sort a Map<Key, Value> by values

Top 3 similar questions with accepted answers are:

+----------------+-----+--------------------+--------------------+------------------+
AcceptedAnswerId|Score| Body| Title| Similarity|
+----------------+-----+--------------------+--------------------+------------------+
 9008961| 2|[I have the follo...|How to sort a Map...|0.9523072691359901|
 47632594| 3|[I have few maps ...|Merge maps and ma...|0.9418623535662379|
 57345829| 1|[I have an unmodi...|How to remove nul...|0.9415684619313417|
+----------------+-----+--------------------+--------------------+------------------+
only showing top 3 rows


---------Pyhton------------
 The highest scored unanswered question is: 
 Score= 1757
 Body= [I want <code>a</code> to be rounded to <em>13.95</em>., The <a href="https://docs.python.org/2/library/functions.html#round" rel="noreferrer"><code>round</code></a> function does not work the way I expected.]
Title= Limiting floats to two decimal points

Top 3 similar questions with accepted answers are:

+----------------+-----+--------------------+--------------------+------------------+
AcceptedAnswerId|Score| Body| Title| Similarity|
+----------------+-----+--------------------+--------------------+------------------+
 50080801| 4|[I'm a beginner i...|docs.python.org i...|0.9117833663532945|
 63925472| 0|[I'm struggling t...|typing: TypeVar v...|0.8906497447264051|
 17044898| 34|[I am looking at ...|Setting the fmt o...|0.8499781599092643|
+----------------+-----+--------------------+--------------------+------------------+
only showing top 3 rows


---------JavaScript------------
 The highest scored unanswered question is: 
 Score= 808
 Body= [How to make an AJAX call using JavaScript, without using jQuery? ]
Title= How to make an AJAX call without jQuery?

Top 3 similar questions with accepted answers are:

+----------------+-----+--------------------+--------------------+------------------+
AcceptedAnswerId|Score| Body| Title| Similarity|
+----------------+-----+--------------------+--------------------+------------------+
 47609427| 0|[I need to pass t...|How to pass a val...| 0.901184468283221|
 32673782| 1|[Is there any dif...|jQuery AJAX metho...|0.8870652915087351|
 9284631| 1|[With jQuery you ...|Can you define an...|0.8659241796446157|
+----------------+-----+--------------------+--------------------+------------------+
only showing top 3 rows